In [1]:
import numpy as np
import sympy
import re
from fractions import Fraction
from tkinter import *

In [7]:
root = Tk()
root.title("Systems of Linear Equations Calculator")
input_values = []
const_values = []
equation_labels = []


#FUNCTIONS
#function to print the grid
def print_grid():
    global rows, cols
    
    rows = int(input_1.get())
    cols = int(input_2.get())

    if 0 < rows and cols <= 5:
        #Creating the label for each variable
        for j in range(cols):
            label = Label(root, text=f"x_{j+1}")
            label.grid(row=1, column=j+1, padx=5, pady=5)

        #Label for const
        Label(root, text="b").grid(row=1, column=cols+1)

        for i in range(rows):
            #Label for each equation
            equation_label = Label(text=i+1)
            equation_label.grid(row=i+5)
            equation_labels.append(equation_label)

            for j in range(cols):
                values = Entry(root, width=10)
                values.grid(row=i+5, column=j+1, padx=5, pady=5)
                input_values.append(values)
            
            # New column for constant entries
            const_entry = Entry(root, width=10)
            const_entry.config(bg="yellow")
            const_entry.grid(row=i+5, column=cols+1, padx=5, pady=5)
            const_values.append(const_entry)

#function to remove the previous matrix
def new_matrix():
    for label in root.grid_slaves(row=1):
        label.grid_forget()

    for values in input_values:
        values.destroy()

    for label in equation_labels:
        label.grid_forget()

    for const_entry in const_values:
        const_entry.grid_forget()

    input_values.clear()
    const_values.clear()
    equation_labels.clear()


# Function to get the value from a specific cell
def get_cell_value(row, col):
    index = row * cols + col
    return input_values[index].get()


# Add this function from your .ipynb to your existing script
def solve_linear_equations(coefficients, constants):
    try:
        # Trying to find a unique solution
        solution = np.linalg.solve(coefficients, constants)
        return solution, "Unique solution"
    except np.linalg.LinAlgError as e:
        if 'Singular matrix' in str(e):
            # This means the matrix is singular and there's no unique solution
            solution, residuals, rank, s = np.linalg.lstsq(coefficients, constants, rcond=None)
            if residuals.size == 0 or np.allclose(residuals, 0):
                return solution, "Infinite solutions"
            else:
                return solution, "No solution"
        else:
            # This is some other kind of linear algebra error we didn't expect
            raise


def format_parametric_solution(solution_str):
    # Split the solution string into individual variable components
    variables = solution_str.split(',')

    # Define a pattern to match the variables like 'x4' and 'x5'
    var_pattern = re.compile(r'x\d+')

    formatted_solution = []

    for i, var in enumerate(variables):
        # Find all free variables in the expression
        free_vars = var_pattern.findall(var)

        # Correctly identify negative and positive floats
        expr = re.sub(r'(?<!\w)-?\d+\.\d+', lambda m: str(Fraction(float(m.group(0))).limit_denominator()), var)

        # If the variable part is a free variable, format it accordingly
        if i + 1 in [int(free_var[1:]) for free_var in free_vars]:
            formatted_solution.append(f"x{i+1} is a free variable")
        else:
            # Remove unnecessary pluses and correct double negatives
            expr = expr.replace(' - -', ' + ').replace('+-', '-')
            formatted_solution.append(f"x{i+1} = {expr.strip()}")

    return formatted_solution


# Add this new function to display the solution
def display_solution(solution, status):
    # Clear the previous solution display if any
    for widget in solution_frame.winfo_children():
        widget.destroy()

    # Display the status
    status_label = Label(solution_frame, text=f"Status: {status}")
    status_label.pack()

    # Display the solution
    if status != "No solution":
        # Ensure solution is passed as a string representation to the formatting function
        formatted_solution_list = format_parametric_solution(str(solution))
        
        # Concatenate the formatted solutions into a single string with line breaks
        formatted_solution_str = "\n".join(formatted_solution_list)
        
        solution_label = Label(solution_frame, text=f"Solution:\n{formatted_solution_str}")
        solution_label.pack()
    else:
        solution_label = Label(solution_frame, text="No solution exists for the given system.")
        solution_label.pack()





# Modify this existing function to solve the equations and display the results
def arr2d_func():
    global coefficient_arr, const_arr
    const_arr = []
    coefficient_arr = []

    for i in range(rows):
        row_values = []
        for j in range(cols):
            value = get_cell_value(i, j)
            try:
                value = Fraction(value)  # Use fractions for exact arithmetic
            except ValueError:
                value = Fraction(0)
            row_values.append(value)
        coefficient_arr.append(row_values)

        const_value = const_values[i].get()
        try:
            const_value = Fraction(const_value)
        except ValueError:
            const_value = Fraction(0)
        const_arr.append(const_value)

    # Convert the arrays to NumPy arrays
    coefficients_array = np.array(coefficient_arr, dtype=float)
    constants_array = np.array(const_arr, dtype=float)

    # Solve the system of equations
    solution, status = solve_linear_equations(coefficients_array, constants_array)

    # Display the results
    display_solution(solution, status)

#GUI INPUTS
# Taking user input for dimensions
Label(text="Enter number of rows:", justify="left").grid(row=0, column=0)
input_1 = Entry(root, width=10)
input_1.grid(row=0, column=1)

Label(text="Enter number of columns:", justify="left").grid(row=0, column=2)
input_2 = Entry(root, width=10)
input_2.grid(row=0, column=3)

#Create matrix button
grid_button = Button(root, text="Create Matrix", command=print_grid, width=15)
grid_button.grid(row=0, column=4)

#New Matrix button
new_button = Button(root, text="Reset", command=new_matrix, width=15)
new_button.grid(row=0, column=5)

#print array
solve_button = Button(root, text="Solve", command=arr2d_func, width=15)
solve_button.grid(row=0, column=6)

# Create a frame to display the solution
solution_frame = Frame(root)
solution_frame.grid(row=8, column=0, columnspan=7, pady=(10, 0))

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10956\1080845659.py", line 150, in arr2d_func
    value = get_cell_value(i, j)
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10956\1080845659.py", line 64, in get_cell_value
    return input_values[index].get()
IndexError: list index out of range
